---
## Python Packages & Directories
---

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u

from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator
plt.rcParams["font.family"] = "Times New Roman"

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind", 10) + sns.color_palette("Paired", 10)[2:] + sns.color_palette("Set2", 10)

# from smh import Session

import spag.read_data as rd
import spag.utils as spagu
from spag.convert import *
import spag.coordinates as coord

script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"


        Use @ instead. [gala.coordinates.sgr]
        Use @ instead. [gala.coordinates.orphan]
        Use @ instead. [gala.coordinates.magellanic_stream]


In [2]:
## Show all columns and rows of the dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

---
## Loading and Modifying Datafile
---

In [109]:
ou2025_df = pd.read_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/ou2025/full_tab_v4.csv")

## Compare the 'ul_[X/H]' to 'exp_ul_[X/H]' columns and the 'ul_[X/Fe]' to 'exp_ul_[X/Fe]' columns
for col in ou2025_df.columns:
    if col.startswith("ul_"):
        flag_col = col
        flag_col_boolean = ou2025_df[flag_col]
        lessthan_col = col.replace("ul_", "exp_ul_")
        lessthan_col_boolean = ou2025_df[lessthan_col].notna() & (ou2025_df[lessthan_col] != '')
        
        comparison = flag_col_boolean == lessthan_col_boolean
        mismatches = ou2025_df.loc[~comparison]

        ## Consolidate the abundance columns
        if len(mismatches) != 0:
            print(col, col.replace("ul_", "exp_ul_"), len(mismatches))
            print(mismatches[[col, col.replace("ul_", "exp_ul_")]])
        else:
            ou2025_df.loc[flag_col_boolean, flag_col] = ou2025_df.loc[flag_col_boolean, col.replace("ul_", "")]
            ou2025_df.loc[~flag_col_boolean, flag_col] = np.nan

ou2025_df.drop(columns=[col for col in ou2025_df.columns if col.startswith("exp_ul_")], inplace=True)
ou2025_df.rename(columns={col: col.replace("ul_", "ul") for col in ou2025_df.columns if col.startswith("ul_")}, inplace=True)

## Rename, Add, and Modify columns
ou2025_df['Source'] = 'Gaia DR3 '+ ou2025_df['Source'].astype(str)
ou2025_df['Loc'] = 'DW'
ou2025_df['Reference'] = 'Ou+2025'
ou2025_df['Ref'] = 'OU25'
ou2025_df.rename(columns={
    'Source': 'Simbad_Identifier',
    'name': 'Name',
    'teff': 'Teff',
    'vt': 'Vmic',
    'etot_teff': 'etot_Teff',
    'etot_vt': 'etot_Vmic',
    'rvobs': 'Vel',
    'corr_[C/Fe]': '[C/Fe]f',
    }, inplace=True)
ou2025_df['[C/Fe]c'] = (ou2025_df['[C/Fe]f'] - ou2025_df['[C/Fe]']).apply(lambda x: normal_round(x, 2))
ou2025_df['ul[C/Fe]f'] = np.nan
ou2025_df['RA_deg'] = np.nan
ou2025_df['DEC_dms'] = np.nan

## Fill the NaN values in the RA and DEC columns
for idx, row in ou2025_df.iterrows():
    if pd.isna(row['RA_deg']) and pd.notna(row['RA_hms']):
        ## pad RA_hms with leading zeros
        if len(row['RA_hms']) == 10:
            row['RA_hms'] = '0' + row['RA_hms']
            ou2025_df.at[idx, 'RA_hms'] = row['RA_hms']
        row['RA_deg'] = coord.ra_hms_to_deg(row['RA_hms'], precision=6)
        ou2025_df.at[idx, 'RA_deg'] = row['RA_deg']

    if pd.isna(row['DEC_deg']) and pd.notna(row['DEC_dms']):
        row['DEC_deg'] = coord.dec_dms_to_deg(row['DEC_dms'], precision=2)
        ou2025_df.at[idx, 'DEC_deg'] = row['DEC_deg']

    if pd.isna(row['RA_hms']) and pd.notna(row['RA_deg']):
        row['RA_hms'] = coord.ra_deg_to_hms(float(row['RA_deg']), precision=2)
        ou2025_df.at[idx, 'RA_hms'] = row['RA_hms']

    if pd.isna(row['DEC_dms']) and pd.notna(row['DEC_deg']):
        row['DEC_dms'] = coord.dec_deg_to_dms(float(row['DEC_deg']), precision=2)
        ou2025_df.at[idx, 'DEC_dms'] = row['DEC_dms']

In [110]:
# Categorize columns & reorder dataFrame
columns = list(ou2025_df.columns)
aux_cols = [
    'Reference','Ref','Name','Simbad_Identifier','Loc','RA_hms','DEC_dms','RA_deg','DEC_deg',
    'lambda','beta','comp','Vel','vhel','vhelcorr','Teff','logg','mh','Vmic',
    'etot_Teff','etot_logg','etot_mh','etot_Vmic','UT-DATE','UT-TIME','BINNING','EXPTIME','SLITSIZE',
]
carbon_cols = [col for col in columns if "[C/" in col]
xh_cols = [col for col in columns if col.startswith("[") and col.endswith("/H]") and col not in carbon_cols]
ul_xh_cols = [col for col in columns if col.startswith("ul[") and col.endswith("/H]") and col not in carbon_cols]
e_xh_cols = [col for col in columns if col.startswith("e_[") and col.endswith("/H]") and col not in carbon_cols]
xfe_cols = [col for col in columns if col.startswith("[") and col.endswith("/Fe]") and col not in carbon_cols]
ul_xfe_cols = [col for col in columns if col.startswith("ul[") and col.endswith("/Fe]") and col not in carbon_cols]
e_xfe_cols = [col for col in columns if col.startswith("e_[") and col.endswith("/Fe]") and col not in carbon_cols]
xy_cols = [col for col in columns if (col.startswith("[") and ("/" in col)) and (col not in xh_cols + xfe_cols + carbon_cols)]
remaining_cols = [col for col in columns if col not in aux_cols + carbon_cols + xh_cols + ul_xh_cols + e_xh_cols + xfe_cols + ul_xfe_cols + e_xfe_cols + xy_cols]

ordered_cols = aux_cols + carbon_cols + xh_cols + ul_xh_cols + e_xh_cols + xfe_cols + ul_xfe_cols + e_xfe_cols + xy_cols + remaining_cols
ou2025_df = ou2025_df[ordered_cols]

In [111]:
## Rename the Abundance columns from ions to elements
for col in ou2025_df.columns:
    if '/H]' in col:
        prefix = col.split("[")[0] + "["
        ion = col.split("[")[1].split("/")[0]
        suffix = "/" + col.split("/")[1]

        if len(ion.split(" ")) > 1:
            if ion.split(" ")[1] == "I":
                element = ion.split(" ")[0]
            elif ion.split(" ")[1] == "II":

                if get_default_ion(ion.split(" ")[0]) == 2:
                    element = ion.split(" ")[0]
                elif get_default_ion(ion.split(" ")[0]) == 1:
                    element = "".join(ion.split(" "))
                else:
                    raise ValueError(f"Unknown ionization state for {ion}")
            else:
                element = ion
        else:
            element = ion
        
        # print(col, prefix+element+suffix)
        ou2025_df.rename(columns={col: prefix+element+suffix}, inplace=True)
    
    elif '/Fe]' in col:
        prefix = col.split("[")[0] + "["
        ion = col.split("[")[1].split("/")[0]
        suffix = "/" + col.split("/")[1]

        if len(ion.split(" ")) > 1:
            if ion.split(" ")[1] == "I":
                element = ion.split(" ")[0]
            elif ion.split(" ")[1] == "II":

                if get_default_ion(ion.split(" ")[0]) == 2:
                    element = ion.split(" ")[0]
                elif get_default_ion(ion.split(" ")[0]) == 1:
                    element = "".join(ion.split(" "))
                else:
                    raise ValueError(f"Unknown ionization state for {ion}")
            else:
                element = ion
        else:
            element = ion

        # print(col, prefix+element+suffix)
        ou2025_df.rename(columns={col: prefix+element+suffix}, inplace=True)

    elif '/' in col:
        prefix = col.split("[")[0] + "["
        ion1 = col.split("[")[1].split("/")[0]
        ion2 = col.split("/")[1].split("]")[0]
        suffix = "]" + col.split("]")[1]

        if len(ion1.split(" ")) > 1:
            if ion1.split(" ")[1] == "I":
                element1 = ion1.split(" ")[0]
            elif ion1.split(" ")[1] == "II":

                if get_default_ion(ion1.split(" ")[0]) == 2:
                    element1 = ion1.split(" ")[0]
                elif get_default_ion(ion1.split(" ")[0]) == 1:
                    element1 = "".join(ion1.split(" "))
                else:
                    raise ValueError(f"Unknown ionization state for {ion}")
            else:
                element1 = ion1
        else:
            element1 = ion1

        if len(ion2.split(" ")) > 1:
            if ion2.split(" ")[1] == "I":
                element2 = ion2.split(" ")[0]
            elif ion2.split(" ")[1] == "II":

                if get_default_ion(ion2.split(" ")[0]) == 2:
                    element2 = ion2.split(" ")[0]
                elif get_default_ion(ion2.split(" ")[0]) == 1:
                    element2 = "".join(ion2.split(" "))
                else:
                    raise ValueError(f"Unknown ionization state for {ion}")
            else:
                element2 = ion2
        else:
            element2 = ion2

        # print(col, prefix+element1+"/"+element2+suffix)
        ou2025_df.rename(columns={col: prefix+element1+"/"+element2+suffix}, inplace=True)

In [112]:
# display(ou2025_df[[col for col in ou2025_df.columns if "C/" in col] + ['logg', '[Fe/H]']])
# display(ou2025_df)
display(list(ou2025_df.columns))

['Reference',
 'Ref',
 'Name',
 'Simbad_Identifier',
 'Loc',
 'RA_hms',
 'DEC_dms',
 'RA_deg',
 'DEC_deg',
 'lambda',
 'beta',
 'comp',
 'Vel',
 'vhel',
 'vhelcorr',
 'Teff',
 'logg',
 'mh',
 'Vmic',
 'etot_Teff',
 'etot_logg',
 'etot_mh',
 'etot_Vmic',
 'UT-DATE',
 'UT-TIME',
 'BINNING',
 'EXPTIME',
 'SLITSIZE',
 '[C/Fe]',
 'e_[C/Fe]',
 'ul[C/Fe]',
 '[C/Fe]f',
 '[C/H]',
 'e_[C/H]',
 'ul[C/H]',
 '[C/Fe]c',
 'ul[C/Fe]f',
 '[Na/H]',
 '[Mg/H]',
 '[Al/H]',
 '[Si/H]',
 '[Ca/H]',
 '[Sc/H]',
 '[Ti/H]',
 '[Cr/H]',
 '[CrII/H]',
 '[Mn/H]',
 '[Co/H]',
 '[Ni/H]',
 '[Zn/H]',
 '[Sr/H]',
 '[Y/H]',
 '[Zr/H]',
 '[Ba/H]',
 '[La/H]',
 '[Eu/H]',
 '[Dy/H]',
 '[Fe/H]',
 'ul[Na/H]',
 'ul[Mg/H]',
 'ul[Al/H]',
 'ul[Si/H]',
 'ul[Ca/H]',
 'ul[Sc/H]',
 'ul[Ti/H]',
 'ul[Cr/H]',
 'ul[CrII/H]',
 'ul[Mn/H]',
 'ul[Co/H]',
 'ul[Ni/H]',
 'ul[Zn/H]',
 'ul[Sr/H]',
 'ul[Y/H]',
 'ul[Zr/H]',
 'ul[Ba/H]',
 'ul[La/H]',
 'ul[Eu/H]',
 'ul[Dy/H]',
 'e_[Na/H]',
 'e_[Mg/H]',
 'e_[Al/H]',
 'e_[Si/H]',
 'e_[Ca/H]',
 'e_[Sc/H]',
 'e_[

In [113]:
ou2025_df.to_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/ou2025/ou2025-yelland.csv", index=False)